In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display

import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

## Auto Ownership

In [ ]:
base_hh = pd.read_csv(f"{base_dir}/final_households.csv",
                       index_col='household_id', 
                       usecols=['household_id','auto_ownership','home_zone_id'])

build_hh = pd.read_csv(f"{build_dir}/final_households.csv", 
                       index_col='household_id', 
                       usecols=['household_id','auto_ownership','home_zone_id'])


In [ ]:
print(f"Filtering households by MAZ.\nOriginal HHs: {len(base_hh)}")
base_hh = base_hh[single_filter_mazs(base_hh.home_zone_id)]
build_hh = build_hh[single_filter_mazs(build_hh.home_zone_id)]
print(f"HHs after filtering: {len(base_hh)}")

In [ ]:
df = pd.crosstab(base_hh.auto_ownership, 
                 build_hh.auto_ownership, 
                 rownames=['Base'],
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df

In [ ]:
diffs = base_hh[['auto_ownership']].merge(
    build_hh[['auto_ownership']],
    how='left',
    left_index=True,
    right_index=True,
    suffixes=('_base','_build')
    )
diffs = diffs.auto_ownership_build - diffs.auto_ownership_base
summary_text = f"""**Auto ownership changes:**
    Households with fewer autos owned in build: {len(diffs[diffs<0])}
    Households with the same autos owned: {len(diffs[diffs==0])}
    Households with more autos owned in build: {len(diffs[diffs>0])}
"""
display(Markdown(summary_text.replace("\n","<br>")))

## Work from Home

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','work_from_home','home_zone_id'])

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','work_from_home','home_zone_id'])

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = pd.crosstab(base_per[base_per.is_worker].work_from_home, 
                 build_per[build_per.is_worker].work_from_home, 
                 rownames=['Base'], 
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df

## Workplace Location Changes

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','is_worker','workplace_zone_id','home_zone_id'])

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','is_worker','workplace_zone_id','home_zone_id'])

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_worker].merge(build_per[build_per.is_worker],
                                        how='left',
                                        left_index=True,
                                        right_index=True, 
                                        suffixes=('_base','_build'))

df = (df.workplace_zone_id_base == df.workplace_zone_id_build).value_counts()
df.index = df.index.map({True:'Unchanged',False:'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## School Location Changes

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','is_student','school_zone_id','home_zone_id'])
build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','is_student','school_zone_id','home_zone_id'])

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_student].merge(build_per[build_per.is_student],
                                         how='left',
                                         left_index=True,
                                         right_index=True,
                                         suffixes=('_base','_build'))

df = (df.school_zone_id_base == df.school_zone_id_build).value_counts()
df.index = df.index.map({True:'Unchanged',False:'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Transit Subsidy

In [ ]:
usecols = ['person_id','transit_pass_ownership','transit_pass_subsidy', 'home_zone_id']
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=usecols)

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=usecols)

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
# Note: subsidy does not appear to be conditioned on ownership
df = base_per[['transit_pass_subsidy']].merge(
    build_per[['transit_pass_subsidy']],
    how='left',
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build')
)

In [ ]:
df0 = pd.crosstab(
    base_per.transit_pass_subsidy,
    build_per.transit_pass_subsidy,
    rownames=['base'],
    colnames=['build'],
    margins=True,
    margins_name='Total'
)
df0

In [ ]:
df = (df.transit_pass_subsidy_base == df.transit_pass_subsidy_build).value_counts()
df.index = df.index.map({True: 'Unchanged', False: 'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Transit Pass Ownership

In [ ]:
df = base_per[['transit_pass_ownership']].merge(
    build_per[['transit_pass_ownership']],
    how='left',
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build')
)

In [ ]:
df0 = pd.crosstab(
    base_per.transit_pass_ownership,
    build_per.transit_pass_ownership,
    rownames=['base'],
    colnames=['build'],
    margins=True,
    margins_name='Total'
)
df0

In [ ]:
df = (df.transit_pass_ownership_base == df.transit_pass_ownership_build).value_counts()
df.index = df.index.map({True: 'Unchanged', False: 'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Free Parking

In [ ]:
usecols = ['person_id', 'is_worker', 'free_parking_at_work', 'workplace_zone_id']
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=usecols)

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=usecols)

In [ ]:
print(f"Filtering persons by workplace MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.workplace_zone_id)]
build_per = build_per[single_filter_mazs(build_per.workplace_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_worker].merge(build_per[build_per.is_worker],
                                        how='left',
                                        left_index=True,
                                        right_index=True, 
                                        suffixes=('_base','_build'))

In [ ]:
df0 = pd.crosstab(
    base_per[base_per.is_worker].free_parking_at_work,
    build_per[build_per.is_worker].free_parking_at_work,
    rownames=['base'],
    colnames=['build'],
    margins=True,
    margins_name='Total'
)
df0

In [ ]:
df = (df.free_parking_at_work_base == df.free_parking_at_work_build).value_counts()
df.index = df.index.map({True:'Unchanged',False:'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Telecommute Frequency

In [ ]:
usecols = ['person_id', 'is_worker', 'telecommute_frequency', 'workplace_zone_id']
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=usecols)

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=usecols)

# nicer names, please. grab the first char to get magnitudes.
tc_labels = {
    'No_Telecommute': '0 (No Telecommute)',
    '1_days_week': '1 Day per Week',
    '2_3_days_week': '2-3 Days per Week',
    '4_days_week': '4 Days per Week'
}

base_per.telecommute_frequency = base_per.telecommute_frequency.map(tc_labels)
build_per.telecommute_frequency = build_per.telecommute_frequency.map(tc_labels)

In [ ]:
print(f"Filtering persons by workplace MAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.workplace_zone_id)]
build_per = build_per[single_filter_mazs(build_per.workplace_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_worker].merge(build_per[build_per.is_worker],
                                        how='left',
                                        left_index=True,
                                        right_index=True, 
                                        suffixes=('_base','_build'))

In [ ]:
xtab = pd.crosstab(
    df.telecommute_frequency_base,
    df.telecommute_frequency_build,
    rownames=['base'],
    colnames=['build'],
    margins=True,
    margins_name='Total'
).sort_index()
xtab

In [ ]:
df1 = df.loc[:, ["telecommute_frequency_base", "telecommute_frequency_build"]]

base_tc_magnitude = df1.telecommute_frequency_base.str[0]
build_tc_magnitude = df1.telecommute_frequency_build.str[0]

df1.loc[base_tc_magnitude == build_tc_magnitude, 'case'] = 'Unchanged'
df1.loc[base_tc_magnitude > build_tc_magnitude, 'case'] = 'Decreased'
df1.loc[base_tc_magnitude < build_tc_magnitude, 'case'] = 'Increased'

fig = vh.create_pie_chart(df1.case.value_counts().to_frame().sort_index(), ["count"])
fig.show()

In [ ]:
summary_text = f"""**Telecommute frequency changes:**
    Workers with decreased telecommuting: {len(df1[df1.case=='Decreased'])}
    Workers with unchanged telecommuting: {len(df1[df1.case=='Unchanged'])}
    Workers with increased telecommuting: {len(df1[df1.case=='Increased'])}
"""
display(Markdown(summary_text.replace("\n","<br>")))

## Transponder Ownership

In [ ]:
base_hh = pd.read_csv(f"{base_dir}/final_households.csv", 
                      index_col='household_id', 
                      usecols=['household_id','transponder_ownership','home_zone_id'])

build_hh = pd.read_csv(f"{build_dir}/final_households.csv", 
                       index_col='household_id', 
                       usecols=['household_id','transponder_ownership', 'home_zone_id'])


In [ ]:
print(f"Filtering households by MAZ. Original HHs: {len(base_hh)}")
base_hh = base_hh[single_filter_mazs(base_hh.home_zone_id)]
build_hh = build_hh[single_filter_mazs(build_hh.home_zone_id)]
print(f"HHs after filtering: {len(base_hh)}")

In [ ]:
df = pd.crosstab(base_hh.transponder_ownership,
                 build_hh.transponder_ownership, 
                 rownames=['Base'],
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df